# Mathematical Notes for Breathing Rate Estimation

This notebook provides the mathematical background for the FFT-based pipeline used in 
`compute_breathing_rate_final.py`.

It is intended as a lesson-style explanation: step-by-step derivation of the equations,
their purpose, and why they are appropriate for this physiological signal processing task.


## Contents

1. FFT power spectrum with window normalization  
2. Frequency resolution and zero-padding  
3. Parabolic peak interpolation  
4. Why FFT is effective for BPM estimation  


## 1. FFT Power Spectrum with Window Normalization

Given a uniformly sampled signal \(x[n]\), we apply a Hann window \(w[n]\) and compute the FFT:

\[
X(f) = \text{FFT}\{x[n] \cdot w[n]\}
\]

The power spectrum is then:

\[
P(f) = \frac{|X(f)|^2}{\sum w[n]^2 \cdot f_s}
\]

This normalization ensures that power estimates are consistent across windows of different
lengths and correctly scaled to sampling frequency.


## 2. Frequency Resolution and Zero-Padding

For an FFT of length \(N\) at sampling rate \(f_s\), the frequency resolution is:

\[
\Delta f = \frac{f_s}{N}
\]

- Larger \(N\) (longer windows) → finer resolution.  
- For short windows, zero-padding increases the number of FFT bins, which refines the grid
  on which peak interpolation is performed.  
- Zero-padding does *not* create new information, but improves the stability of peak estimation.


## 3. Parabolic Peak Interpolation

Suppose the FFT magnitude spectrum has a maximum at bin \(k\), with neighbors
\(y_{-1}, y_0, y_{+1}\).

We can estimate the true peak frequency by fitting a parabola through these three points.

The offset of the vertex from the center bin is:

\[
\delta = \frac{1}{2} \cdot \frac{y_{-1} - y_{+1}}{y_{-1} - 2y_0 + y_{+1}}
\]

The refined peak frequency is then:

\[
f^\* = f_k + \delta \cdot \Delta f
\]

This allows sub-bin frequency resolution, which is essential when analyzing short (15 s) windows.


## 4. Why FFT is Effective for BPM Estimation

- The IBI-derived signal contains a **periodic oscillation** due to respiration.
- In the frequency domain, periodic oscillations appear as **narrowband peaks**.
- By restricting analysis to the respiratory band (0.1–0.5 Hz), we ensure only
  physiologically plausible peaks are considered.
- Converting the dominant peak frequency to breaths per minute is straightforward:

\[
\text{BPM} = 60 \cdot f^\*
\]


## References

- Harris, F. J. (1978). *On the use of windows for harmonic analysis with the discrete Fourier transform.* Proceedings of the IEEE.  
- Welch, P. (1967). *The use of fast Fourier transform for the estimation of power spectra.* IEEE Trans. Audio Electroacoustics.  
- Quinn, B. G. (1994). *Estimating frequency by interpolation using Fourier coefficients.* IEEE Trans. Signal Processing.  
- Jacobsen, E., & Kootsookos, P. (2007). *Fast, accurate frequency estimators.* IEEE Signal Processing Magazine.  
- Task Force of the ESC (1996). *Heart rate variability: Standards of measurement, physiological interpretation and clinical use.* Circulation.
